> Go is a *statically typed*, *garbage collected*, *compiled* language that is capable of producing highly concurrent, thread-safe programs that will scale as needed

*Static typing* 
- Each statement is checked at compile time.
- Data types are bound to variables (instead of to values as in dynamically typed languages like PHP) and must be defined when declaring variables

*Garbage collection*
- Developers don't need to explicitly allocate and deallocate memory for storing variables. Instead, developers use a garbage collector to monitor the program's memory usage.
- The garbage collector periodically determine which memory is still in use and which is not, and releases the memory no longer being used by the program
- A garbage collector makes it easier to write code but may have a performance cost; As you generate more memory usage(or "garbage"), the garbage collector need to spend time releasing that memory later. So, excessive garbage can lead to poorly performing programs.

*Compilation*
- One of the original design goals behind the Go language was fast compile times.
- The `go build` command is used to compile the source code.
- Errors in the code such as mismatch types are caught at compile time. unlike interpreted languages that will continue to execute every line till it encounters the error and raises and exception.

**Keywords Operators and Delimiters**
- keywords are reserved and cannot be used as an identifier.
- Follow these links to find the list of [keywords](https://go.dev/ref/spec#Keywords), [operators](https://go.dev/ref/spec#Operators) and [assignment operators](https://go.dev/ref/spec#assign_op)

### Numbers
Go has two types of numeric types; 
- *Architecture-independent* type which means regardless of the architecture you compile for, the type will the correct size, bytes, for your type.
- *Implementation-specific* type where the byte size of the numeric type can vary based on the architecture the program is built for.
- Architecture independent numeric types include; `uint8`, `uint16`, `uint32`, `uint64`, `int8`, `int16`, `int32`, `int64`, `float32`, `float64`, `complex64`, `complex128`, `byte`, `rune`.
- `byte` is alias for `uint8` and `rune` is alias for `int32`
- Architecture implementation specific types include; `uint`, `int`, `uintptr`
- size of `uint` will be 32 bits on 32-bit architecture and 64 bits on 64-bit architecture.
- list of these types and their ranges can be found [here](https://go.dev/ref/spec#Numeric_types)

Picking the correct type usually has more to do with performance for the target architecture you are programming for than the size of the data you are working with.
- For integer data, it is common to use the implementation types like `int` and `uint`. This typically results in the fastest processing speed for your target architecture
- If you know you wont exceed a specific size range, picking an architecture independent type can both increase speed and decrease memory usage.

*OverFlow & Wraparound* \
When performing mathematical calculations,  there is a potential to either "overflow" a number or "wraparound" a number.
- At compile time, if the compiler can determine a value will be too large to hold in the data type specified, it will throw an overflow error, indicating that the value calculated to store is too large for the data type you specified.


In [ ]:
// example of overflow

package main

import "fmt"

func main () {
	var maxUint uint8 = 260 // uint8's max value is 255
	fmt.Println("value: ", maxUint)
}

// this code will return an error at compile time

In [ ]:
// example of wraparound

package main

import "fmt"

func main () {
  var maxInt uint8 = 255 // uint8's max value is 255
  fmt.Println("value: ", maxInt+5)
}

// this code will compile successfully but the will print 4 instead of 260

- Go does not saturate variable during mathematical operations such as addition or multiplication. In languages that saturate, if you have a uint8 with a max value of 255 and add 1, the value is still the max(saturated) value of 255. In Go however, it always wraps around.
- Understanding the boundaries of your data helps you avoid potential bugs in your program in the future.
- read more about saturation arithmetic on [wikipedia](https://en.wikipedia.org/wiki/Saturation_arithmetic)

### Strings

Go uses double quotes and backticks for strings. \
Backticks create raw string literals. Double quotes create interpreted string literals.

- Any characters except newline and an unescaped double quotes may appear within the quotes.
- Backslashes (escape sequences) have no special meaning inside raw string literals.
- Raw string literals are useful for creating multiline templates and mock JSON data for testing, etc

### UTF-8
Go supports UTF-8 characters out of the box. [more on utf-8 on wikipedia](https://en.wikipedia.org/wiki/UTF-8)

In [ ]:
func main() {
	a := "Hello, 世界" // hello world (English + Chinese)
	fmt.Println(a)
}

### Runes
- A `rune` is an alias for int32 and is used to represent single characters. 
- A rune can be made up of 1, 2 or 3 int32 values, allowing for both single and multi-byte characters.
A rune can be defined using a single quote (') character

In [ ]:
func main () {
	a := 'A'

	fmt.Printf("%v (%T)\n", a, a) // outputs value of a and type of a.
}

// when printed as a value, a rune will be printed as an int32 (which is the UTF-8 encoding of the character)

**Iterating over UTF-8 characters**
- When iterating over a string and trying to access individual characters, we may encounter issues with UTF-8 characters

In [ ]:
func main () {
	a := "Hello, 世界" // 9 characters
	for i := 0; i < len(a); i++ {
		fmt.Printf("%d: %s\n", i, string(a[i]))
	}
}

// the output is unpredictable when iterating over UTF-8 string of characters.

Notice  that the unexpected characters in the output of the above code are printed for index 7-12. This is because multiple int32s make up the rune for the chinese characters but we are only taking part of it. \
In simpler terms; \
The issue is that the string contains both ASCII and UTF-8 characters. ASCII characters are represented by 1 byte each in Go. UTF-8 characters, on the other hand, are represented by multiple bytes (e.g., 3 bytes for Chinese characters like 世 or 界).
The len() function in the for loop simply counts the number of bytes, in the argument passed to it. This means the loop breaks the string into 1-byte pieces, regardless of whether the character is ASCII or UTF-8.
The string() function, much like len(), tries to interpret each byte individually as a single character (a 32-bit rune). This works fine for 1-byte ASCII characters, but it cannot correctly interpret UTF-8 characters because they are split into their individual bytes. As a result, the Chinese characters are broken into 3 invalid pieces each and printed as unexpected or garbled characters.

When iterating through each character in a string in Go, the proper way is to use the range keyword in the loop;

In [ ]:
func main () {
	a := "Hello, 世界" // 9 characters
  for i, c := range a {
    fmt.Printf("%d: %s \n", i, string(c))
  }
}

// the range keyword ensures that we use the proper index and length of int32s to capture the proper rune value

### Variables
- declaration: `var <name> <type>`
  - `var i int`
  - `var b bool`
- assignment: `<name> = <value>`
  - `i = 26`
  - `b = false`

**Zero Values**
- Variables when declared without values, assume a zero value of the same type as the variable. Complex types such as a `struct` have a zero value that is composed of the type's individual fields' zero values. 
- Certain types in Go such as maps, interfaces and pointers have no obvious zero values, hence their zero values is `nil`.

// zero value cheatsheet

```go
  var s string // ""
  var r rune // 0
  var bt byte // 0
  var i int // 0
  var ui uint // 0
  var c complex64 // 0
  var b bool // false
  var arr [2]int // [0 0]
  var obj struct {
    b bool
    arr [2]int
  } // {false [0 0]}
  var si []int // nil
  var ch chan string // nil
  var mp map[int]string // nil
  var fn func() // nil
  var ptr *string // nil
  var all any // nil
```

- When declaring a variable, we often want to initialize it with a value; `var f float64 = 3.14`
- When initializing a variable at declaration time,  we can use the `:=` operator; `i := 42`
- When using the `:=` operator, go infers the type of the variable from the value it is being assigned.
- For most types, inferring the type is no trouble. For numbers, the compiler has to guess the type, and by default, assumes that the variable is of type int or float64 (if there is a decimal point in it)
- If for numbers you desire some other type other that int or float64,  use the `var <name> <type> [= <value>]` to declare the variable or use variable casting like so; `i := uint32(42)`, `f := float32(3.14)`

Go allows you to assign values to multiple variables in one line. this approach is mostly used for  capturing the result of a function call that returns multiple values
```go
func main() {
  x, y, z := "Some text", 3.14, 34
  // notice that different data types can be assigned on a single line without any issues.
}
```
While using this very useful technique, remember that code readability must not be sacrificed for fewer lines of code.

- In Go, unused variables, and imports are not allowed. Also, all returned values from a function must be captured.
- Discard variables not needed by identifying them with the blank identifier _ (underscore) \
Let's see more clearly; If `MyFunction()` returns 3 variables, we must capture all of them
```go
func main () {
  a, b, c := MyFunction() // capturing 3 variables returned by MyFunction.
  // If we need just he 3rd value, we can use the blank identifier to clear off the first 2 like so;
  _, _, z := MyFunction() // Now the compiler will discard the first 2 returned values.
}
```


### Constants

- Constants in Go can only be a character, string, boolean or numeric value.
- We use the `const` keyword to define variables; `const myVar = 23.2`
- Values that can be modified such as maps and slices cannot be constants, and neither can the result of a function call.

In [ ]:
func main() {
	const gopher = func()string{
		return "Gopher"
	}() // results in compile-time error

	const names = []string {"Amie", "Carlos", "Ndzi"} // results in compile-time error
}

**Typed Constants**
- If a contant is decleared with a type, it will be that exact type. It can therefore only operate with values of that type. 

In [ ]:
import "fmt"

const (
	leapYear = int32(366)
)
func main(){
	hour := 24
	fmt.Println(hour * leapYear) // cannot operate int with int32
}

**Untyped Constants (Inferred typing)**
- Constants can be `untyped`, meaning it inferred it's type from value assigned to it. This can be useful when working with with numbers such as integers.
- Untyped contants are explicitly converted. Typed constants are not. Example below makes it clear;

In [ ]:
const (
	year = 365 //untyped
	leapYear = int32(366) //typed
)

func main(){
	hours := 24
	minutes := int32(60)

	fmt.Println(hours * year) //int * untyped
	fmt.Println(minutes * year) //int32 * untyped
	fmt.Println(minutes * leapYear) //int32 * int32
}

**Type Inference**
- Remember that untyped constants and variables will be converted to the type it is combined with for any operation for that type. In other terms, both untyped variables and contants assume the most suitable types for current operation.

### Naming Identifiers

Idetifier names have the following rules;
- They are case sensitive
- They cannot be reserved words
- Can only be made of letters, numbers and underscores
- They cannot begin with a number.

**Naming Style**
- The most important thing with style is consistency and acceptance in a team.
- It is common in Go to use very terse (or short) variable names. Given the choice
between using userName and user for a variable, it would be idiomatic to choose user.
- Scope also plays a role in the terseness of the variable name. The rule is that the smaller
the scope the variable exists in, the smaller the variable name.
- Examples of conventional<->unconventional identifier names: `userName`<->`user_name` (prefers PascalCase/camelCase), `i`<->`index`(prefers shorter names depending on scope), `serverHTTP`<->`serverHttp` (prefers capitalizing acronyms) 

- Go allows identifier names to conflict with package names. However, from point in the file where the variable is declared, the package with the same name is no longer accessible. The best way to resolve this is to rename the variable. Ofcourse you can also use import alias for the package.

**Eporting through capitalization** \
If an identifier name begins with a capital letter, it is exported (made available outside the package is it declared in). Identifiers that start with lower case letters are not exported.

### Printing and Formatting
The fmt package provides a number of functions for printing and formatting in Go. The style is similar to c-progamming but easier. \


In [ ]:
! go doc fmt

The fmt documentation show many identical functions. The core of these functions is to generate formatted strings based on user input. Lets address a few.
- Functions that start with `Sprint` all return formatted strings
- Functions that start with `Print` all print formatted strings on the standard ouput
- Functions that start with `Fprint` all print formatted strings on the provided io.Writter. (`go doc fmt.Fprint` for more info)

- Most fmt's Printing and Sprinting functions do not add new lines to the end of the string. For example, you can see results from 2 Print functions on the same line. New lines can be added with the `\n` escape sequence.
- The `Println` function automatically adds a new line.
- Multiple arguments can be passed (separated by commas)
- All foratting functions ending in `f`(such as `fmt.Sprintf`) allows for the use of formatting verbs. Verbs create different styles of formatting and are usually preceeded by either `%` or `\` 
- Formatting verbs begin with a `%` character. Escape sequences begin with a `\` character.
- There may be times that you want to use those characters as themselves in the formatted string. To do this, each character needs to be escaped by repeating itself.

**Formatting strings**
- The most common verbs for string formatting are `%s` and `%q`. The former print the string as it is while the later also prints the string as it is but with quotes around it. \
**Formatting integers**
- All integer-based types (int, int32, int64, ...) abide to the same formatting rules. The verb `%d` is used to print integers. `%+d` prints the integer with a sign(negative numbers will always show their sign regardless.) \
**Integer padding**
- The `%d` verb can be modified to add padding either to the right or left of the integer to print. In either case, the padding is considered a minimum padding. So, if the number is larger than the minimum padding, it is not padded.
- The width of the padding is declared by using an integer in front of the `d` in the verb. \
**Formatting floats** 
- Use `%f` verb
- Number of decimal places can be specified like so; `%.2f` to specify 2 decimal places and so on


In [ ]:
// Examples
func main(){
	fmt.Printf("%q \n", "quoted string")
	fmt.Printf("Singed integer: %+d \n", +2334)
	fmt.Printf("padded with 5 spaces %5d \n", 123)
	fmt.Printf("padded with 5 zeros %05d \n", 123)
	fmt.Printf("padded to the right with 5 spaces %-5d \n", 123)
	fmt.Printf("3 decimal places: %.3f \n", 23.234325)
}

- `%T` prints the type of the value
- `%v` prints the value of the variable
- `%+v` prints more information about the variable if possible. For example, printing a struct with `%v` will print only the field values whereas `%+v` will print the field names and values.
- `%#v` prints the value in native Go-syntax format.

In [ ]:
package main

import "fmt"

type User struct {
	Name string
	Age int
}

func main(){
	u := User{
		Name: "Kimbi",
		Age: 20,
	}

	fmt.Printf("type of u: %T \n", u)
	fmt.Printf("value of u: %v \n", u)
	fmt.Printf("details of u: %+v \n", u)
	fmt.Printf("go-syntax represenation of u: %#v \n", u)
}

- When you use a formatting verb incorrectly, such as trying to format a string with a %d
verb, there will be no error to respond to. Instead, Go prints error information into the
formatted string.
- To avoid using the wrong verbs, and other issues, you should always run `go vet` on
your code

**Explicit Augmented Indexes** \
The default behaviour of fmt's functions is for each formatting verb to format successive arguments passed into the call. However, you can use `[N]` to specify which argument to use. See example below. \
This specification also allows you to use the same variable multiple times.

In [ ]:
func main() {
	fmt.Printf("in succession: %s, %s, %s \n", "one", "two", "three")
	fmt.Printf("specifying arguments: %[2]s, %[1]s \n", "one", "two")
	fmt.Printf("repeated: %[1]s, %[1]s \n", "one")
}

### Converting strings to and from numbers
- The `strconv` package has methods to explicitly allow conversion between strings and numbers.
- The most common strconv methods are `ParseInt`, `ParseUint`, `ParseFloat` and `Atoi`
- Methods like ParseInt requires as parameter, a the base we will be using and the bit size being expected from the input. The default values of 0 and  64 respectively are good enough for most use cases. Check the docs for strconv and it's methods `go doc strconv`, `go doc strconv.ParseUint`

In [ ]:
package main

import (
	"fmt"
	"strconv"
	"log"
)

func main(){
	// string to integer
	i, err := strconv.ParseInt("-42", 0, 64)
	if err != nil {
		log.Fatal(err)
	}
	fmt.Printf("str: '-42' --> int: %d \n", i)
	
	// parsing a float
	j, err := strconv.ParseFloat("43.234", 64)
	if err != nil {
		log.Fatal(err)
	}
	fmt.Printf("str: '43.234' --> int: %f \n", j)
}
